# TRABAJO PRÁCTICO 9 - NAIVE BAYES

In [11]:
#librerias
import os
import urllib.request
import tarfile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter

from sklearn.model_selection import train_test_split

# Procesamiento del catálogo

#### Descarga del catálogo

In [12]:
# Se colocan las URLs de los archivos a descargar
url_catalog = "https://web.csc.gob.ar/~jzuloaga/epub/catalog.csv"
url_tar = "https://web.csc.gob.ar/~jzuloaga/epub/compressed.tar"

# Se colocan nombres a los archivos
file_catalog = "catalog.csv"
file_tar = "compressed.tar"

# Se descargan solo si no existen
if not os.path.exists(file_catalog):
    print("Descargando catalog.csv...")
    urllib.request.urlretrieve(url_catalog, file_catalog)

if not os.path.exists(file_tar):
    print("Descargando compressed.tar...")
    urllib.request.urlretrieve(url_tar, file_tar)

print("Descarga de archivos completa.")

# Descompresion del catalogo
# Se descomprimen los archivos
if not os.path.exists("compressed"):
    with tarfile.open(file_tar, "r") as tar:
        tar.extractall("compressed")
        print("Archivos extraídos en la carpeta 'compressed'\n")
else:
    print("La carpeta 'compressed' ya existe, no se vuelve a extraer.\n")

Descargando catalog.csv...
Descargando compressed.tar...
Descarga de archivos completa.


C:\Users\solek\AppData\Local\Temp\ipykernel_21156\48687499.py:24: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall("compressed")


Archivos extraídos en la carpeta 'compressed'



#### Cargar el catálogo y explorar el contenido de sus columnas.  ¿Qué representa cada una?

In [13]:
# Se carga el catalogo
df_catalog = pd.read_csv("catalog.csv")

# Se muestran las columnas del catalogo
print("Columnas del catálogo:")
print(df_catalog['Géneros'])

# Se muestran las primeras filas del catalogo
print("\nPrimeras filas del catálogo:")
print(df_catalog['Estado'])

Columnas del catálogo:
0                                 Otros
1        Aventuras, Fantástico, Juvenil
2           Ciencias sociales, Historia
3                    Historia, Memorias
4         Biografía, Historia, Memorias
                      ...              
69188                        Fantástico
69189                             Otros
69190                             Humor
69191                 Intriga, Policial
69192                         Filosofía
Name: Géneros, Length: 69193, dtype: object

Primeras filas del catálogo:
0        Disp.
1        Disp.
2        Disp.
3        Disp.
4        Disp.
         ...  
69188    Disp.
69189     Ver.
69190     Ver.
69191    Disp.
69192    Disp.
Name: Estado, Length: 69193, dtype: object


#### Filtrar las entradas del catálogo, de manera de quedarse solamente con los libros en idioma español.

In [14]:
spanish_books = df_catalog[df_catalog['Idioma'] == 'Español']

#### Limitar las entradas del catálogo a las que tenga su correspondiente libro digital.

In [15]:
digital_books = df_catalog[ df_catalog["Estado"] == "Disp."]

# Definición de las clases

#### Analizar la distribución de libros por categoría.

In [16]:
# Se separan los géneros y se cuentan individualmente
all_genres = df_catalog['Géneros'].dropna().str.split(r',\s*')
genre_counter = Counter([g for sublist in all_genres for g in sublist])

# Se muestran los 10 primeros géneros más frecuentes
genre_distribution = pd.Series(genre_counter).sort_values(ascending=False)
genre_distribution.head(10)

Intriga            9572
Policial           7754
Histórico          7208
Otros              6419
Romántico          6349
Drama              6296
Fantástico         6129
Ciencia ficción    4994
Realista           4705
Historia           4407
dtype: int64

#### Eliminar el género Otros por ser una categoría redundante.

In [17]:
# Se eliminan los libros con únicamente el género "Otros"
df_catalog = df_catalog[~df_catalog['Géneros'].str.fullmatch(r'Otros', na=False)]

# Se elimina el género "Otros" de los demás libros
df_catalog['Géneros'] = (
    df_catalog['Géneros']
    .str.replace(r',?\s*Otros', '', regex=True)
    .str.strip(', ')
)

#### Proponer y justificar un criterio para elegir un único género cuando un libro tenga varios.

In [18]:
# Dado que por lo general el primer género es el más relevante asociado al contenido del libro, 
# se considerará este cuando un libro tenga varios géneros.
df_catalog['Género_principal'] = df_catalog['Géneros'].str.split(',').str[0].str.strip()

# Se elimina la columna "Géneros" del catálogo
df_catalog.drop(columns=['Géneros'], inplace=True)

#### Reportar la distribución final de libros por categoría

In [19]:
# Se cuenta la cantidad de apariciones de cada género en la columna de 'Género_principal'
final_distribution = df_catalog['Género_principal'].value_counts()

# Se muestran los 10 primeros géneros más frecuentes
final_distribution.head(10)

Género_principal
Intriga              7321
Drama                6006
Histórico            5377
Ciencia ficción      4715
Fantástico           4448
Aventuras            4220
Policial             4103
Historia             3354
Realista             2934
Ciencias sociales    2821
Name: count, dtype: int64

#### Separar los libros para definir conjuntos de entrenamiento y testeo utilizando las proporciones 75/25. Fijar la semilla para reproducibilidad utilizando su núumero de padrón.

In [20]:
# Se coloca el número de padrón
student_id = 104241

# Se separan en datos de entrenamiento y testeo
train_set, test_set = train_test_split(df_catalog, test_size=0.25, random_state=student_id)

print('Tamaño del catálogo completo:', df_catalog.shape)
print('Tamaño del conjunto de entrenamiento:', train_set.shape)
print('Tamaño del conjunto de testeo:', test_set.shape)

Tamaño del catálogo completo: (64895, 16)
Tamaño del conjunto de entrenamiento: (48671, 16)
Tamaño del conjunto de testeo: (16224, 16)
